In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project="vaibhav-gee")

In [3]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
District = ee.FeatureCollection("projects/vaibhav-gee/assets/DISTRICT_BOUNDARY")

In [5]:
map.addLayer(District, {}, "District")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
MOD44B = ee.ImageCollection("MODIS/006/MOD44B").select("Percent_Tree_Cover")

In [7]:
Tree_Cover_2001 = MOD44B.filterDate("2001-01-01","2001-12-31").mean().clip(District)
Tree_Cover_2017 = MOD44B.filterDate("2017-01-01","2017-12-31").mean().clip(District)
viz_tc01 = {"min":0,"max":78.0,"palette":['#fef6e3','#fde5a8','#fdd26e','#e6f598','#c7e89e','#91cf60','#66bd63','#3d9b4f','#1a7a3b','#00441b']}
viz_tc17 = {"min":0,"max":78.0,"palette":['#fef6e3','#fde5a8','#fdd26e','#e6f598','#c7e89e','#91cf60','#66bd63','#3d9b4f','#1a7a3b','#00441b']}
map.addLayer(Tree_Cover_2001,viz_tc01,"Tree_Cover_2001")
map.addLayer(Tree_Cover_2017,viz_tc17,"Tree_Cover_2017")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())

        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)

District_TC_2001 = compute_zonal_stats(Tree_Cover_2001, District, scale=250)
District_TC_2017 = compute_zonal_stats(Tree_Cover_2017, District, scale=250)

In [9]:
map.addLayer(District_TC_2001,{},"District_TC_2001")
map.addLayer(District_TC_2017,{},"District_TC_2017")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
MCD15A3H = ee.ImageCollection("MODIS/061/MCD15A3H").select("Lai")

In [11]:
LAI_2003 = MCD15A3H.filterDate("2003-01-01","2003-12-31").mean().clip(District)
LAI_2025 = MCD15A3H.filterDate("2025-01-01","2025-12-31").mean().clip(District)
viz_tc03 = {"min":1.1,"max":44.0,"palette":['#f7fcf5', '#e7f6e2', '#ceecc7', '#aedea7', '#88cd86', '#5db96b', '#37a055', '#1b843f', '#00682a', '#00441b']}
viz_tc25 = {"min":1.1,"max":43.4,"palette":['#f7fcf5', '#e7f6e2', '#ceecc7', '#aedea7', '#88cd86', '#5db96b', '#37a055', '#1b843f', '#00682a', '#00441b']}
map.addLayer(LAI_2003,viz_tc03,"LAI_2003")
map.addLayer(LAI_2025,viz_tc25,"LAI_2025")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())
        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)

District_LAI_2003 = compute_zonal_stats(LAI_2003, District, scale=500)
District_LAI_2025 = compute_zonal_stats(LAI_2025, District, scale=500)

In [13]:
map.addLayer(District_LAI_2003,{},"District_LAI_2003")
map.addLayer(District_LAI_2025,{},"District_LAI_2025")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [14]:
MOD11A1 = ee.ImageCollection("MODIS/061/MOD11A1").select("LST_Day_1km")

In [15]:
LST_2004 = MOD11A1.filterDate('2004-01-01', '2004-12-31').mean().multiply(0.02).subtract(273.15).clip(District)
LST_2023 = MOD11A1.filterDate('2023-01-01', '2023-12-31').mean().multiply(0.02).subtract(273.15).clip(District)
vis_lst04 = {'min': 15, 'max': 45, 'palette': ['#2166ac', '#4393c3', '#92c5de', '#d1e5f0', '#fef8b8', '#fee08b', '#fdae61', '#f46d43', '#d73027', '#a50026']}
vis_lst23 = {'min': 15, 'max': 45, 'palette': ['#2166ac', '#4393c3', '#92c5de', '#d1e5f0', '#fef8b8', '#fee08b', '#fdae61', '#f46d43', '#d73027', '#a50026']}
map.addLayer(LST_2004, vis_lst04, "LST_2004")
map.addLayer(LST_2023, vis_lst23, "LST_2023")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [16]:
def compute_zonal_stats(image, features, scale=500):
    """
    Compute zonal statistics for each district
    """
    def calc_stats(feature):
        clipped = image.clip(feature.geometry())
        stats = clipped.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                ee.Reducer.min(), '', True).combine(
                ee.Reducer.max(), '', True).combine(
                ee.Reducer.stdDev(), '', True),
            geometry=feature.geometry(),
            scale=scale,
            maxPixels=1e13
        )
        return feature.set(stats)

    return features.map(calc_stats)

# Compute zonal statistics for LST
District_LST_2004 = compute_zonal_stats(LST_2004, District, scale=1000)
District_LST_2023 = compute_zonal_stats(LST_2023, District, scale=1000)

In [17]:
map.addLayer(District_LST_2004,{},"District_LST_2004")
map.addLayer(District_LST_2023,{},"District_LST_2023")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [18]:
geemap.ee_to_geojson(District_TC_2001, filename='District_TC_2001.geojson')
print("Exported: District_TC_2001.geojson")

Exported: District_TC_2001.geojson


In [19]:
geemap.ee_to_geojson(District_TC_2017, filename='District_TC_2017.geojson')
print("Exported: District_TC_2017.geojson")

Exported: District_TC_2017.geojson


In [20]:
geemap.ee_to_geojson(District_LAI_2003, filename='District_LAI_2003.geojson')
print("Exported: District_LAI_2003.geojson")

Exported: District_LAI_2003.geojson


In [21]:
geemap.ee_to_geojson(District_LAI_2025, filename='District_LAI_2025.geojson')
print("Exported: District_LAI_2025.geojson")

Exported: District_LAI_2025.geojson


In [22]:
geemap.ee_to_geojson(District_LST_2004, filename='District_LST_2004.geojson')
print("Exported: District_LST_2004.geojson")

Exported: District_LST_2004.geojson


In [23]:
geemap.ee_to_geojson(District_LST_2023, filename='District_LST_2023.geojson')
print("Exported: District_LST_2023.geojson")

Exported: District_LST_2023.geojson
